### converting pixeldata to png (수정 필요)

0) 3D 엑스레이 장비 데이터용

1) 작업 폴더 복사

2) 복사한 폴더에 image 폴더 생성

3) image 폴더에 이미지 파일 저장

4) 이미지 변환 완료 후 binary 폴더 삭제

5) 파일 이름 변경

In [3]:
import shutil # 파일, 폴더 복사 모듈
import cv2 # openCV 모듈 
import os
import numpy as np
import pandas as pd
import natsort # 숫자로 된 파일명 정렬 모듈

In [ ]:
# 엑스레이 이미지 파일 변환ver0.1

df = pd.read_excel('C:\\Users\\user\\Desktop\\(국토부과제 2차년도) Runlist_E3S 690 (Mix)_Rev1.4.xlsx', header=None) # 엑셀 파일 위치 지정

dir_working = 'C:\\Users\\user\\Desktop\\image\\엑스레이' # 촬영 데이터가 모여져 있는 폴더 경로

i = 1456 # 파일 명 찾는 가변 변수
j = 0 # Serial Number 붙일 때 활용하는 가변 변수

dir_working_copy = dir_working + '\\이미지 변환'

if not os.path.exists(dir_working_copy):
    shutil.copytree(dir_working, dir_working_copy) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.

filelist = os.listdir(dir_working_copy)

for file in filelist:
    os.chdir(dir_working_copy) # change directory
    # read image as binary
    read_binary = open(dir_working_copy + '/' + file, "rb") # 1. 파일 경로, 2. read + binary (rb)
    
    img = np.fromfile(read_binary, dtype = 'uint16', sep = "", count = (896*760), offset = 0) # dtype, count는 상황에 맞게 수정 필요    
    read_binary.close()
    img = np.reshape(img, [760,896]) # 810 -> height, 896 -> width
    markfile = df.values[i][11] + "_" + str(j) + '.png'
    
    cv2.imwrite(markfile, img)
    os.chdir(dir_working_copy)

    print(file)
    print(df.values[i][4])
    print(markfile)

    if os.path.exists(dir_working_copy + '/' + file):
        os.remove(dir_working_copy +'/'+ file) # 이미지 변환이 끝난 raw 파일은 필요 없으므로 삭제함.
        
    j = j + 1

    if (j == 9):
        i = i + 1
        j = 0
        


In [ ]:
# 엑스레이 이미지 파일 변환ver0.2 -> 이미지 짤리는 현상 보안

import shutil # 파일, 폴더 복사 모듈
import cv2 # openCV 모듈 
import os
import numpy as np
import pandas as pd

df = pd.read_excel('C:\\Users\\user\\Desktop\\test.xlsx', header=None) # 엑셀 파일 위치 지정

dir_working = 'C:\\Users\\user\\Desktop\\photo\\X-ray Data\\Mix\\20220512' # 촬영 데이터가 모여져 있는 폴더 경로

i = 9441 # 파일 명 찾는 가변 변수 - 시작 번호
height = 760
width = 896

dir_working_copy = dir_working + '\\이미지 변환'
dir_working_result = dir_working + '\\result'

if not os.path.exists(dir_working_copy):
    shutil.copytree(dir_working, dir_working_copy) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.
if not os.path.exists(dir_working_result):
    os.mkdir(dir_working_result)

filelist = os.listdir(dir_working_copy)

for file in filelist:

    os.chdir(dir_working_copy) # change directory
    
    # read image as binary
    read_binary = open(dir_working_copy+'/'+file, "rb") # 1. 파일 경로, 2. read + binary (rb)
    img = np.fromfile(read_binary, dtype='uint16', sep="")
    read_binary.close()

    img = np.reshape(img, [img.shape[0]//width, width]) # height -> 전체 count // width, 896 -> width
    
    # image 파일 저장 이름 
    markfile = df.values[i][1] + '.png'
    
    cv2.imwrite(markfile, img)

    # 파일 변환 확인용
    print(file)
    print(markfile)
    print(df.values[i][0])

    # image detection code
    img_detection = cv2.imread(markfile, cv2.IMREAD_COLOR) # 이미지 불러오기

    # 이미지 그레이 변환 -> 
    gray = cv2.cvtColor(img_detection, cv2.COLOR_BGR2GRAY) 

    # 가우시안 블러 사용 -> 이미지를 부드럽게 하기에 배경과 원하지 않는 부분을 제거하는 효과
    blur = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=0)

    # 임계값 설정
    redlalwlt, th1 = cv2.threshold(blur, 150, 255, cv2.THRESH_BINARY_INV)
    binary = cv2.bitwise_not(th1)

    # contours -> 3차원
    contours, hierarchy = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours_xy = np.array(contours) # 윤곽선 최소 좌표를 구하기 위해 변형

    value = list()

    for x in range(len(contours_xy)): 
        for y in range(len(contours_xy[x])):
            value.append(contours_xy[x][y][0][1]) # height 값

    y_min = min(value) # 최소 좌표

    
    # height 시작 위치 조정
    if (y_min - 55 <= 0):
        y_min = 0
    else:
        y_min = y_min - 55
    
    img = img[y_min:y_min+height, :]

    
    # 이미지 변환이 끝난 raw 파일은 필요 없으므로 삭제함.
    if os.path.exists(dir_working_copy + '/' + file):
        os.remove(dir_working_copy +'/'+ file)
    
    os.chdir(dir_working_result)
    cv2.imwrite(markfile, img)
    

    # 가변 변수 1 증가  
    i = i + 1

# 마지막 레코드 -> i 시작 번호에 복사해서 붙여넣기
print(i)


In [ ]:
# 실사 이미지 파일 변환 ver0.1

df = pd.read_excel('C:\\Users\\user\\Desktop\\국토부과제_2차년도_엑스레이 이미지 데이터 DB_ver0.3.xlsx')

dir_working = 'C:\\Users\\user\\Desktop\\실사이미지\\Round 0'

filelist = os.listdir(dir_working)

filelist = natsort.natsorted(filelist)

for file in filelist:
    newname = df.values[9 * (int(file.split(".")[0]) + 443)][-1][:-2]
    os.rename(dir_working + "/" + file, dir_working + "/" + newname + ".jpg")

In [ ]:
# 실사 이미지 파일 변환 ver0.2 -> 새로운 파일에 데이터 저장 후 진행

df = pd.read_excel('C:\\Users\\user\\Desktop\\(국토부과제 2차년도) Runlist_E3S 690 (Mix)_Rev1.2.xlsx', header=None)
dir_working = 'C:\\Users\\user\\Desktop\\image'

filelist = os.listdir(dir_working)

filelist = natsort.natsorted(filelist)

for i in range(0, len(filelist) - 1):
    for j in range(0, 9):
        print(j)
    newname = df.values[i][1]
    print(filelist[i])
    print(df.values[i][0])
    print(newname)
    # os.rename(dir_working + "/" + filelist[i], dir_working + "/" + newname + ".jpg")
    

In [ ]:
# 실사 이미지 파일 변환 ver0.3

import shutil # 파일, 폴더 복사 모듈
import os
import numpy as np
import pandas as pd
import natsort # 숫자로 된 파일명 정렬 모듈

df = pd.read_excel('C:\\Users\\user\\Desktop\\(국토부과제 2차년도) Runlist_E3S 690 (Mix)_Rev1.4.xlsx', header=None)
dir_working = 'C:\\Users\\user\\Desktop\\image\\실사'
dir_working_result = dir_working + '/' + dir_working.split('\\')[-1]

if not os.path.exists(dir_working_result):
    shutil.copytree(dir_working, dir_working_result) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.

i = 1456 # 파일 명 찾는 가변 변수

filelist = os.listdir(dir_working_result)

filelist = natsort.natsorted(filelist)

for file in filelist:
    os.chdir(dir_working_result) # change directory
    
    newname = df.values[i][11] + ".jpg"
    os.rename(dir_working_result + "/" + file, dir_working_result + "/" + newname)
    print(file)
    print(df.values[i][3])
    print(newname)
    i = i + 1

print(i)

In [ ]:
# 실사 이미지 파일 변환 ver0.3

import shutil # 파일, 폴더 복사 모듈
import os
import numpy as np
import pandas as pd
import natsort # 숫자로 된 파일명 정렬 모듈

df = pd.read_excel('C:\\Users\\user\\Desktop\\test.xlsx', header=None)

dir_working = 'C:\\Users\\user\\Desktop\\photo\\Photo Data\\Mix\\20220512'
dir_working_result = dir_working + '/' + dir_working.split('\\')[-1]

if not os.path.exists(dir_working_result):
    shutil.copytree(dir_working, dir_working_result) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.

i = 1049 # 파일 명 찾는 가변 변수

filelist = os.listdir(dir_working_result)

filelist = natsort.natsorted(filelist)

for file in filelist:
    
    os.chdir(dir_working_result) # change directory
    
    newname = df.values[i*9][1][:-2] + ".jpg"
    os.rename(dir_working_result + "/" + file, dir_working_result + "/" + newname)
    print(file)
    print(df.values[i*9][0])
    print(newname)
    i = i + 1

print(i)

In [ ]:
# 이미지 파일 장비 이름 변경 코드
import os

dir_working = '\\\\172.16.150.135\\2_팀 공유 폴더\\1) X-Ray 촬영 이미지 & 관리 문서\\2) 3D X-Ray 이미지_SSTLabs\\SSTLabs 3D 촬영 이미지\\20220510~20220512 촬영데이터\\엑스레이 이미지_신버전\\20220510'

filelist = os.listdir(dir_working)

for file in filelist:
    newname = file[0:3] + str(690) + file[6:]
    os.rename(dir_working + "/" + file, dir_working + "/" + newname)
    print(file)
    print(newname)

In [ ]:
# 실사 이미지 네이밍 규칙 변경 v1.0
import shutil # 파일, 폴더 복사 모듈
import os
import numpy as np
import pandas as pd
import natsort # 숫자로 된 파일명 정렬 모듈

df = pd.read_excel('C:\\Users\\user\\Desktop\\국토부과제_2차년도_엑스레이 이미지 데이터 DB_ver0.10.xlsx', header=None)

dir_working = '\\\\172.16.150.135\\2_팀 공유 폴더\\1) X-Ray 촬영 이미지 & 관리 문서\\2) 3D X-Ray 이미지_SSTLabs\\SSTLabs 3D 촬영 이미지\\20220412~20220413 촬영데이터\\개봉 실사 이미지_신버전\\Round 0 확인'

dir_working_result = dir_working + '/' + dir_working.split('\\')[-1]

if not os.path.exists(dir_working_result):
    shutil.copytree(dir_working, dir_working_result) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.
    
i = 3968 # 파일 명 찾는 가변 변수
j = 0

filelist = os.listdir(dir_working_result)

filelist = natsort.natsorted(filelist)

for file in filelist:
    os.chdir(dir_working_result) # change directory
    
    newname = df.values[i + j*9][10][:-2] + ".jpg"
    os.rename(dir_working_result + "/" + file, dir_working_result + "/" + newname)
    print(file)
    print(newname)
    print("-------------")
    j = j + 1

print(i + j * 9)

In [1]:
# 엑스레이 이미지 네이밍 규칙 변경 v1.0
import shutil # 파일, 폴더 복사 모듈
import os
import numpy as np
import pandas as pd
import natsort # 숫자로 된 파일명 정렬 모듈

df = pd.read_excel('C:\\Users\\user\\Desktop\\국토부과제_2차년도_엑스레이 이미지 데이터 DB_ver0.10.xlsx', header=None)

dir_working = '\\\\172.16.150.135\\2_팀 공유 폴더\\1) X-Ray 촬영 이미지 & 관리 문서\\2) 3D X-Ray 이미지_SSTLabs\\SSTLabs 3D 촬영 이미지\\20220412~20220413 촬영데이터\\엑스레이 이미지_신버전\\Round 1 확인'

dir_working_result = dir_working + '/' + dir_working.split('\\')[-1]

if not os.path.exists(dir_working_result):
    shutil.copytree(dir_working, dir_working_result) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.
    
i = 1 # 파일 명 찾는 가변 변수

filelist = os.listdir(dir_working_result)

filelist = natsort.natsorted(filelist)

for file in filelist:
    os.chdir(dir_working_result) # change directory
    
    newname = df.values[i][10] + ".jpg"
    os.rename(dir_working_result + "/" + file, dir_working_result + "/" + newname)
    print(file)
    print(newname)
    print("-------------")
    i = i + 1

print(i)